# Data Cleaning and Feature Preparation:

First of all we have done a data cleaning of the database to have everything in a proper way to use an algorithm of machine learning.

After that, we created the features that are going to be used for the model.
We took into account for each team (away and home) the rank they have in the league at the moment of the game, also the points they obtained out of the total possible (relative strenght). For example, being in the tenth matchday a team can be in the sixth position of the championship, but have a difference of 5 points with the fifteenth team, this means that there is an small difference in the mid-table, so, it is really usefull to take into account not only the position in the championship, but the relative strenght too.
As well, a team attack and defend different when play at home or away, knowing that the public do a lot cheering for their team and the stadiums differ from each other in the exact measures and the quality of the field (especially in the older games), so, we took into account the amount of goals scored by a team at home and away, and also the amount of goals received at home and away.
Also, it is important the form of a team over the year, taking into account that the teams usually play only in weekends and the type of injuries that happen in football, made us to put as a feature the streak of the team in the last five games, because more it is too much and less games is a streak too short.
The other features, are basic feautures such as teams, points of each team, score, matchday, division, etc.

In [17]:
import sqlite3
import pandas as pd
from datetime import datetime

In [18]:
# Connect to the SQLite database
conn = sqlite3.connect('../laliga.sqlite')

# Query to select the relevant columns from the matches table
query = '''
SELECT score, home_team, away_team, season, division, matchday
FROM Matches
'''

# Run the query and store the results in a DataFrame
matches_df = pd.read_sql_query(query, conn)

# Drop rows with null scores
matches_df.dropna(subset=['score'], inplace=True)

# Apply the date formatting function to the date column
#matches_df['date'] = matches_df['date'].apply(parse_date_mdy_format)

# Close the database connection
conn.close()

matches_df.head()  # Display the first few rows of the DataFrame to verify

,score,home_team,away_team,season,division,matchday
0,2:3,Arenas Club,Athletic Madrid,1928-1929,1,1
1,3:2,Espanyol,Real Unión,1928-1929,1,1
2,5:0,Real Madrid,Catalunya,1928-1929,1,1
3,1:1,Donostia,Athletic,1928-1929,1,1
4,0:2,Racing,Barcelona,1928-1929,1,1


In [19]:
# Helper function to determine match outcomes 
def determine_outcome(home_goals, away_goals):
    if home_goals > away_goals:
        return 'W', 'L'
    elif home_goals < away_goals:
        return 'L', 'W'
    else:
        return 'T', 'T'

In [20]:
def calculate_single_season_division_standings(season, division, matches_df):
    # Filter matches for the given season and division
    season_matches = matches_df[(matches_df['season'] == season) & (matches_df['division'] == division)].sort_values('matchday')
    matchdays = season_matches['matchday'].unique()

    # Initialize the standings dict with teams
    teams = pd.unique(season_matches[['home_team', 'away_team']].values.ravel('K'))
    standings_tracker = {team: {'GF_home': 0, 'GA_home': 0, 'GF_away': 0, 'GA_away': 0, 'W': 0, 'L': 0, 'T': 0, 'PTS': 0, 'last_5': []} for team in teams}

    # Function to determine match outcome
    def determine_outcome(home_goals, away_goals):
        if home_goals > away_goals:
            return 'W', 'L'
        elif home_goals < away_goals:
            return 'L', 'W'
        return 'T', 'T'

    # Function to calculate relative strength
    def calculate_relative_strength(standings_df, matchday):
        # Calculate the maximum possible points so far for each team
        max_points_so_far = matchday * 3
        # Calculate the relative strength for each team as the percentage of points obtained out of the total possible
        standings_df['relative_strength'] = standings_df['PTS'] / max_points_so_far * 100
        return standings_df

    # List to collect matchday standings
    all_standings = []

    # Process each matchday
    for matchday in matchdays:
        matchday_matches = season_matches[season_matches['matchday'] == matchday]
        for index, match in matchday_matches.iterrows():
            home_team, away_team = match['home_team'], match['away_team']
            home_goals, away_goals = map(int, match['score'].split(':'))
            home_outcome, away_outcome = determine_outcome(home_goals, away_goals)

            # Update goals for and against at home and away
            standings_tracker[home_team]['GF_home'] += home_goals
            standings_tracker[away_team]['GF_away'] += away_goals
            standings_tracker[home_team]['GA_home'] += away_goals
            standings_tracker[away_team]['GA_away'] += home_goals

            # Update last 5 matches
            if matchday > 1:  # Only update if it's not the first matchday
                standings_tracker[home_team]['last_5'].insert(0, home_outcome)
                standings_tracker[away_team]['last_5'].insert(0, away_outcome)

            # Ensure last_5 lists do not exceed 5 matches
            standings_tracker[home_team]['last_5'] = standings_tracker[home_team]['last_5'][:5]
            standings_tracker[away_team]['last_5'] = standings_tracker[away_team]['last_5'][:5]

            # Update wins, losses, ties, and points
            if home_goals > away_goals:  # Home win
                standings_tracker[home_team]['W'] += 1
                standings_tracker[home_team]['PTS'] += 3
                standings_tracker[away_team]['L'] += 1
            elif home_goals < away_goals:  # Away win
                standings_tracker[away_team]['W'] += 1
                standings_tracker[away_team]['PTS'] += 3
                standings_tracker[home_team]['L'] += 1
            else:  # Tie
                standings_tracker[home_team]['T'] += 1
                standings_tracker[home_team]['PTS'] += 1
                standings_tracker[away_team]['T'] += 1
                standings_tracker[away_team]['PTS'] += 1

        # Calculate goal difference for each team
        for team in teams:
            standings_tracker[team]['GD'] = standings_tracker[team]['GF_home'] + standings_tracker[team]['GF_away'] - \
                                             standings_tracker[team]['GA_home'] - standings_tracker[team]['GA_away']

        # Create standings DataFrame for the current matchday
        matchday_standings = (pd.DataFrame.from_dict(standings_tracker, orient='index')
                                .reset_index()
                                .rename(columns={'index': 'team'}))
        matchday_standings['matchday'] = matchday
        matchday_standings['season'] = season
        matchday_standings['division'] = division

        # Sort standings
        matchday_standings.sort_values(by=['PTS', 'GD', 'GF_home', 'GF_away'], ascending=[False, False, False, False], inplace=True)
        matchday_standings['rank'] = matchday_standings.reset_index(drop=True).index + 1

        # Calculate the relative strength
        matchday_standings = calculate_relative_strength(matchday_standings, matchday)

        # Append to the list
        all_standings.append(matchday_standings)

    # Concatenate all matchday standings
    final_standings = pd.concat(all_standings, ignore_index=True)
    # Reorder columns
    final_standings = final_standings[['season', 'division', 'matchday', 'rank', 'team', 'GD', 'GF_home', 'GA_home', 'GF_away', 'GA_away', 'W', 'L', 'T', 'PTS', 'last_5', 'relative_strength']]
    
    return final_standings

# Use this function to generate the standings for a given season and division
#calculate_single_season_division_standings('2020-2021', 1, matches_df)

def calculate_all_seasons_divisions_standings(matches_df):
    # Initialize the final DataFrame
    final_all_standings = pd.DataFrame()

    # Process each season and division without explicit loops
    for (season, division), group_df in matches_df.groupby(['season', 'division']):
        season_division_standings = calculate_single_season_division_standings(season, division, group_df)
        final_all_standings = pd.concat([final_all_standings, season_division_standings], ignore_index=True)

    return final_all_standings

features_df = calculate_all_seasons_divisions_standings(matches_df)

In [21]:
def encode_match_outcome(score):
    home_goals, away_goals = map(int, score.split(':'))
    if home_goals > away_goals:
        return '1'  # Home win
    elif home_goals < away_goals:
        return '2'  # Away win
    else:
        return 'X'  # Draw

# Apply the encoding function to the score column
matches_df['outcome'] = matches_df['score'].apply(encode_match_outcome)

In [22]:
print(matches_df.columns.to_list())

features_df.drop(['W', 'L', 'T'], axis=1, inplace=True)
print(features_df.columns.to_list())

['score', 'home_team', 'away_team', 'season', 'division', 'matchday', 'outcome']
['season', 'division', 'matchday', 'rank', 'team', 'GD', 'GF_home', 'GA_home', 'GF_away', 'GA_away', 'PTS', 'last_5', 'relative_strength']


In [23]:
# First merge for Home Team features
matches_df = pd.merge(
    left=matches_df,
    right=features_df,
    how='left',
    left_on=['season', 'division', 'matchday', 'home_team'],
    right_on=['season', 'division', 'matchday', 'team'],
    suffixes=('', '_home')
)

# Rename the merged columns for the Home Team
home_feature_columns = {
    'GD_home' : 'GD_HomeTeam',
    'GF_home_home': 'GFH_HomeTeam',
    'GA_home_home': 'GAH_HomeTeam',
    'GF_away_home': 'GFA_HomeTeam',
    'GA_away_home': 'GAA_HomeTeam',
    'PTS_home': 'PTS_HomeTeam',
    'last_5_home': 'last_5_HomeTeam',
    'relative_strength_home': 'relative_strength_HomeTeam',
    'rank_home': 'rank_HomeTeam'
}
matches_df.rename(columns=home_feature_columns, inplace=True)

# Second merge for Away Team features
matches_df = pd.merge(
    left=matches_df,
    right=features_df,
    how='left',
    left_on=['season', 'division', 'matchday', 'away_team'],
    right_on=['season', 'division', 'matchday', 'team'],
    suffixes=('', '_away')
)

# Rename the merged columns for the Away Team
away_feature_columns = {
    'GD_away' : 'GD_AwayTeam',
    'GF_home_away': 'GFH_AwayTeam',
    'GA_home_away': 'GAH_AwayTeam',
    'GF_away_away': 'GFA_AwayTeam',
    'GA_away_away': 'GAA_AwayTeam',
    'PTS_away': 'PTS_AwayTeam',
    'last_5_away': 'last_5_AwayTeam',
    'relative_strength_away': 'relative_strength_AwayTeam',
    'rank_away': 'rank_AwayTeam'
}
matches_df.rename(columns=away_feature_columns, inplace=True)


print(matches_df.columns.to_list())

['score', 'home_team', 'away_team', 'season', 'division', 'matchday', 'outcome', 'rank', 'team', 'GD', 'GF_home', 'GA_home', 'GF_away', 'GA_away', 'PTS', 'last_5', 'relative_strength', 'rank_AwayTeam', 'team_away', 'GD_AwayTeam', 'GFH_AwayTeam', 'GAH_AwayTeam', 'GFA_AwayTeam', 'GAA_AwayTeam', 'PTS_AwayTeam', 'last_5_AwayTeam', 'relative_strength_AwayTeam']


In [24]:
matches_df.drop(['home_team', 'away_team', 'team_away'], axis=1, inplace=True)

matches_df['season_start'] = matches_df['season'].apply(lambda x: int(x.split('-')[0]))
min_season = matches_df['season_start'].min()
matches_df['season_since_start'] = matches_df['season_start'] - min_season

print(matches_df.columns.to_list())

['score', 'season', 'division', 'matchday', 'outcome', 'rank', 'team', 'GD', 'GF_home', 'GA_home', 'GF_away', 'GA_away', 'PTS', 'last_5', 'relative_strength', 'rank_AwayTeam', 'GD_AwayTeam', 'GFH_AwayTeam', 'GAH_AwayTeam', 'GFA_AwayTeam', 'GAA_AwayTeam', 'PTS_AwayTeam', 'last_5_AwayTeam', 'relative_strength_AwayTeam', 'season_start', 'season_since_start']


In [25]:
def encode_last_5(results_list):
    # Define points for win, draw, loss
    points = {'W': 3, 'D': 1, 'L': 0}
    
    # Calculate the total points for the last 5 matches
    total_points = sum(points[result] for result in results_list if result in points)
    
    return total_points

# Now apply this function to the last_5 column of your DataFrame
matches_df['last_5_AwayTeam'] = matches_df['last_5_AwayTeam'].apply(encode_last_5)
matches_df['last_5_HomeTeam'] = matches_df['last_5'].apply(encode_last_5)

matches_df.drop('last_5', axis=1, inplace=True)

# Testing ML Algorithms:

After dealing with the data to prepare it, we tried different algorithms of machine learning with target of the outcome of the match (who wins or if they tie) and a test size of the 20%, because of the characteristics of the problem the algorithms we tried are a random forest, a gradient boosting, a neural network, a recurrent neural network (RNN) and a long short-term memory (LSTM), that is a type of recurrent neural network.

After trying all of this algorithms we found that all of them work good with our problem because they have almost 70% of accuracy determining the outcome of the matches. But the one who fits better with our problem is the RNN having 70% of accuracy.
This is a good value taking into account that soccer is a very unpredictable sport, so, not having more accuracy is due to that fact. So, we can conclude that this model works good and the features used fit in the model good too.

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encoding categorical features and scaling
label_encoder = LabelEncoder()
matches_df['season'] = label_encoder.fit_transform(matches_df['season'])

# Assuming 'score' is not a feature, and 'PTS' and 'PTS_AwayTeam' are the same, dropping duplicates
matches_df.drop(['score', 'team', 'PTS'], axis=1, inplace=True)

# Split the data into features and target
X = matches_df.drop('outcome', axis=1)
y = matches_df['outcome']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test data
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6828125


In [27]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize the model
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

# Fit the model on the training data
gbm.fit(X_train, y_train)

# Predict on the test data
gbm_predictions = gbm.predict(X_test)

# You can also evaluate the model performance
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, gbm_predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6986458333333333


In [28]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Assuming you have your input features in `X` and labels in `y` as a pandas Series or numpy array

# Step 1: Encode the labels
label_encoder = LabelEncoder()
y_integers = label_encoder.fit_transform(y)
y_encoded = to_categorical(y_integers, num_classes=3)

# Step 2: Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Step 3: Create the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 3 classes for win, draw, loss

# Step 4: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Step 6: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy}")

Epoch 1/50
1200/1200 [==============================] - 1s 775us/step - loss: 2.8435 - accuracy: 0.5314
Epoch 2/50
1200/1200 [==============================] - 1s 785us/step - loss: 1.3755 - accuracy: 0.5809
Epoch 3/50
1200/1200 [==============================] - 1s 780us/step - loss: 1.2708 - accuracy: 0.5958
Epoch 4/50
1200/1200 [==============================] - 1s 784us/step - loss: 1.1568 - accuracy: 0.6071
Epoch 5/50
1200/1200 [==============================] - 1s 778us/step - loss: 1.1241 - accuracy: 0.6090
Epoch 6/50
1200/1200 [==============================] - 1s 776us/step - loss: 1.1022 - accuracy: 0.6084
Epoch 7/50
1200/1200 [==============================] - 1s 760us/step - loss: 1.0550 - accuracy: 0.6124
Epoch 8/50
1200/1200 [==============================] - 1s 782us/step - loss: 0.9824 - accuracy: 0.6203
Epoch 9/50
1200/1200 [==============================] - 1s 791us/step - loss: 0.9505 - accuracy: 0.6278
Epoch 10/50
1200/1200 [==============================] - 1s 782u

In [29]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Step 1: Encode the labels
label_encoder = LabelEncoder()
y_integers = label_encoder.fit_transform(y)
y_encoded = to_categorical(y_integers, num_classes=3)

# Step 2: Normalize the features (assuming X is a DataFrame for this example)
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)

# Step 3: Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Reshape input data to be 3D [samples, timesteps, features] for LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Step 4: Define LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(3, activation='softmax')) # Output layer with 3 units for each class

# Step 5: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 6: Fit the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=2)

# Step 7: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy:.2f}')

Epoch 1/50
1200/1200 - 2s - loss: 0.8066 - accuracy: 0.6396 - val_loss: 0.7414 - val_accuracy: 0.6819 - 2s/epoch - 2ms/step
Epoch 2/50
1200/1200 - 1s - loss: 0.7381 - accuracy: 0.6786 - val_loss: 0.7213 - val_accuracy: 0.6865 - 1s/epoch - 1ms/step
Epoch 3/50
1200/1200 - 1s - loss: 0.7253 - accuracy: 0.6817 - val_loss: 0.7137 - val_accuracy: 0.6828 - 1s/epoch - 1ms/step
Epoch 4/50
1200/1200 - 1s - loss: 0.7221 - accuracy: 0.6813 - val_loss: 0.7151 - val_accuracy: 0.6803 - 1s/epoch - 1ms/step
Epoch 5/50
1200/1200 - 1s - loss: 0.7176 - accuracy: 0.6812 - val_loss: 0.7061 - val_accuracy: 0.6843 - 1s/epoch - 1ms/step
Epoch 6/50
1200/1200 - 1s - loss: 0.7144 - accuracy: 0.6831 - val_loss: 0.7006 - val_accuracy: 0.6911 - 1s/epoch - 1ms/step
Epoch 7/50
1200/1200 - 1s - loss: 0.7114 - accuracy: 0.6852 - val_loss: 0.6983 - val_accuracy: 0.6873 - 1s/epoch - 1ms/step
Epoch 8/50
1200/1200 - 1s - loss: 0.7081 - accuracy: 0.6842 - val_loss: 0.7019 - val_accuracy: 0.6904 - 1s/epoch - 1ms/step
Epoch 9/

In [30]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Step 1: Encode the labels
label_encoder = LabelEncoder()
y_integers = label_encoder.fit_transform(y)
y_encoded = to_categorical(y_integers, num_classes=3)

# Step 2: Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)

# Step 3: Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Reshape input data to be 3D [samples, time_steps, features] for RNN
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Step 4: Create RNN model
model = Sequential()
model.add(SimpleRNN(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(3, activation='softmax')) # Output layer with 3 units for each class

# Step 5: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 6: Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=2)

# Step 7: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy:.2f}')

Epoch 1/50
1200/1200 - 2s - loss: 0.7884 - accuracy: 0.6506 - val_loss: 0.7290 - val_accuracy: 0.6831 - 2s/epoch - 2ms/step
Epoch 2/50
1200/1200 - 1s - loss: 0.7293 - accuracy: 0.6771 - val_loss: 0.7117 - val_accuracy: 0.6854 - 1s/epoch - 942us/step
Epoch 3/50
1200/1200 - 1s - loss: 0.7185 - accuracy: 0.6812 - val_loss: 0.7054 - val_accuracy: 0.6881 - 1s/epoch - 969us/step
Epoch 4/50
1200/1200 - 1s - loss: 0.7127 - accuracy: 0.6837 - val_loss: 0.7006 - val_accuracy: 0.6905 - 1s/epoch - 954us/step
Epoch 5/50
1200/1200 - 1s - loss: 0.7093 - accuracy: 0.6838 - val_loss: 0.7059 - val_accuracy: 0.6843 - 1s/epoch - 962us/step
Epoch 6/50
1200/1200 - 1s - loss: 0.7052 - accuracy: 0.6857 - val_loss: 0.6963 - val_accuracy: 0.6864 - 1s/epoch - 955us/step
Epoch 7/50
1200/1200 - 1s - loss: 0.7024 - accuracy: 0.6873 - val_loss: 0.6940 - val_accuracy: 0.6907 - 1s/epoch - 981us/step
Epoch 8/50
1200/1200 - 1s - loss: 0.6992 - accuracy: 0.6890 - val_loss: 0.6884 - val_accuracy: 0.6923 - 1s/epoch - 958us